In [276]:
import pandas as pd
import ssl 
import numpy as np
import os
import requests
import logging
from bs4 import BeautifulSoup
from pathlib import Path

ssl._create_default_https_context = ssl._create_unverified_context
#p_id='253802'

Table: 'career_avg_batsman'

In [235]:
def career_avg_batsman(p_id):
    career_avg_batsman = pd.read_html("https://stats.espncricinfo.com/ci/engine/player/"+p_id+".html?class=2;template=results;type=batting")[2]
    career_avg_batsman=career_avg_batsman[["Span","Inns","Runs","HS","Ave","SR","100","50","0","4s","6s"]]

    career_avg_batsman.insert(0, "P_id", [p_id], True)

    filepath = Path('Csv/Career_avg_batsman.csv')
    hdr = False  if os.path.isfile(filepath) else True
    career_avg_batsman.to_csv(filepath,  mode='a', index=False, header=hdr)  
    career_avg_batsman


Table: 'vs_country_batsman'

In [236]:
def vs_country_batsman(p_id):
    vs_country_batsman = pd.read_html("https://stats.espncricinfo.com/ci/engine/player/"+p_id+".html?class=2;template=results;type=batting")[3]
    vs_country_batsman=vs_country_batsman[["Grouping","Inns","Runs","HS","Ave","SR"]]

    vs_country_batsman.rename(columns = {'Grouping':'Country'}, inplace = True)
    vs_country_batsman.insert(0, "P_id", p_id, True)

    countries=['v Pakistan', 'v Australia', 'v England', 'v New Zealand', 'v West Indies', 'v Sri Lanka', 'v Bangladesh', 'v South Africa']

    select_vs_country_batsman = vs_country_batsman.loc[ vs_country_batsman['Country'].isin(countries)  ] 

    filepath = Path('Csv/Vs_country_batsman.csv')
    hdr = False  if os.path.isfile(filepath) else True
    select_vs_country_batsman.to_csv(filepath, mode='a', index=False, header=hdr)  
    select_vs_country_batsman

Table: 'home_vs_away_batsman'

In [237]:
def home_vs_away_batsman(p_id):
    home_vs_away_batsman = pd.read_html("https://stats.espncricinfo.com/ci/engine/player/"+p_id+".html?class=2;template=results;type=batting")[3]
    home_vs_away_batsman=home_vs_away_batsman[["Grouping","Inns","Runs","HS","Ave","SR"]]

    home_vs_away_batsman.rename(columns = {'Grouping':'Venue'}, inplace = True)
    home_vs_away_batsman.insert(0, "P_id", p_id, True)

    venues=['home', 'away', 'neutral']

    select_home_vs_away_batsman = home_vs_away_batsman.loc[ home_vs_away_batsman['Venue'].isin(venues)  ] 

    filepath = Path('Csv/Home_vs_away_batsman.csv')
    hdr = False  if os.path.isfile(filepath) else True
    select_home_vs_away_batsman.to_csv(filepath, mode='a', index=False, header=hdr)  
    select_home_vs_away_batsman

Table: 'yearly_stats_batsman'

In [238]:
def yearly_stats_batsman(p_id):
    yearly_stats_batsman = pd.read_html("https://stats.espncricinfo.com/ci/engine/player/"+p_id+".html?class=2;template=results;type=batting")[3]
    yearly_stats_batsman=yearly_stats_batsman[["Grouping","Inns","Runs","HS","Ave","SR"]]

    yearly_stats_batsman.rename(columns = {'Grouping':'Year'}, inplace = True)
    yearly_stats_batsman.insert(0, "P_id", p_id, True)

    years=['home', 'away', 'neutral']

    select_yearly_stats_batsman = yearly_stats_batsman.loc[ yearly_stats_batsman['Year'].str.startswith('year ', na=False)  ] 

    filepath = Path('Csv/Yearly_stats_batsman.csv')
    hdr = False  if os.path.isfile(filepath) else True
    select_yearly_stats_batsman.to_csv(filepath, mode='a', index=False, header=hdr)  
    select_yearly_stats_batsman

In [288]:
def player_overview(p_id):
    # URL of the website to scrape
    url = "https://www.espncricinfo.com/cricketers/x-"+p_id

    # Send an HTTP GET request to the website
    response = requests.get(url)

    # Parse the HTML code using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract the relevant information from the HTML code
    class_grid='ds-grid lg:ds-grid-cols-3 ds-grid-cols-2 ds-gap-4 ds-mb-8'
    div_p = soup.find('div', {'class': class_grid})
    
    my_dict = {}

    class_para = 'ds-text-tight-m ds-font-regular ds-uppercase ds-text-typo-mid3'
    for div in div_p.find_all('div'):
        if ( div.find('p', {'class':class_para}) ) is None:
            continue

        key = div.find('p', {'class':class_para}).text
        value = div.find('span').text
        my_dict[key] = value

    #Adds players' roll ids: (ALL for All rounder), (BAT for Batsman), (BOW for Bowler)
    if 'All' in my_dict.get('Playing Role'):
        my_dict['Roll_id'] = 'ALL'
    elif 'Bowl' in my_dict.get('Playing Role'):
        my_dict['Roll_id'] = 'BOW'
    elif 'Bat' in my_dict.get('Playing Role'):
        my_dict['Roll_id'] = 'BAT'
    else:
        log.error('No matching playing role is found')

    player_overview = pd.DataFrame.from_dict([my_dict])
    player_overview.insert(0, "P_id", p_id, True)

    filepath = Path('Csv/player_overview.csv')
    hdr = False  if os.path.isfile(filepath) else True
    player_overview.to_csv(filepath, mode='a', index=False, header=hdr)
    print(player_overview)

#Main function below:

In [289]:
#253802: Virat Kohli (BAT)
#34102: Rohit Gurunath Sharma (BAT)
#422108: Kannaur Lokesh Rahul (BAT)
#234675: Ravindrasinh Anirudhsinh Jadeja (ALL)
#625371: Hardik Himanshu Pandya (ALL)

player_ids=['253802', '34102', '422108', '234675', '625371']
#player_ids=['625383', '481896', '26421', '234675', '625371']

#Removes garbage files
os.system("rm -f Csv/*_batsman.csv")

for pid in player_ids:
    career_avg_batsman(pid)
    vs_country_batsman(pid)
    home_vs_away_batsman(pid)
    yearly_stats_batsman(pid)
    player_overview(pid)
    time.sleep(1)
    

     P_id    Full Name                      Born       Age   Batting Style  \
0  253802  Virat Kohli  November 05, 1988, Delhi  34y 360d  Right hand Bat   

      Bowling Style      Playing Role Roll_id  
0  Right arm Medium  Top order Batter     BAT  
    P_id              Full Name                                         Born  \
0  34102  Rohit Gurunath Sharma  April 30, 1987, Bansod, Nagpur, Maharashtra   

        Age   Batting Style       Bowling Style      Playing Role Roll_id  
0  36y 184d  Right hand Bat  Right arm Offbreak  Top order Batter     BAT  
     P_id             Full Name                                  Born  \
0  422108  Kannaur Lokesh Rahul  April 18, 1992, Bangalore, Karnataka   

        Age   Batting Style Fielding Position         Playing Role Roll_id  
0  31y 196d  Right hand Bat      Wicketkeeper  Wicketkeeper Batter     BAT  
     P_id                        Full Name  \
0  234675  Ravindrasinh Anirudhsinh Jadeja   

                                        